In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import gptqmodel
dir(gptqmodel)

/Users/atharvabot7/Downloads/MLOPS-Tutorials/Fine-Tuning-Practice/gptq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


/Users/atharvabot7/Downloads/MLOPS-Tutorials/Fine-Tuning-Practice/gptq/lib/python3.11/site-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/Context.cpp:85.)
  self.setter(val)


['BACKEND',
 'BaseQuantizeConfig',
 'GPTQModel',
 'QuantizeConfig',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'adapter',
 'exllama_set_max_input_length',
 'get_best_device',
 'looper',
 'models',
 'nn_modules',
 'os',
 'quantization',
 'utils',
 'version']

In [3]:
from gptqmodel import GPTQModel

In [4]:
model = GPTQModel.load("ModelCloud/Llama-3.2-1B-Instruct-gptqmodel-4bit-vortex-v2.5")

from_quantized: adapter: None


Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 46798.37it/s]

INFO  Loader: Auto dtype (MPS or XPU): `torch.float16`                         
INFO  Estimated Quantization BPW (bits per weight): 4.85 bpw, based on [bits: 4, group_size: 32]



`torch_dtype` is deprecated! Use `dtype` instead!


INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             
INFO  Kernel: candidates -> `[TorchQuantLinear]`                               
INFO  Kernel: selected -> `TorchQuantLinear`.                                  


INFO  Format: Converting `checkpoint_format` from `FORMAT.GPTQ` to internal `FORMAT.GPTQ_V2`.
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Conversion complete: 0.008443832397460938s                       
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             
INFO  Optimize: `TorchQuantLinear` compilation triggered.                      


INFO:tokenicer.tokenicer:Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').


INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

INFO  Model: `generation_config.json` not found. Skipped checking.             
INFO  Kernel: loaded -> `[TorchQuantLinear]`                                   


In [5]:
model

LlamaGPTQ(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 2048)
      (layers): ModuleList(
        (0-15): 16 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): TorchQuantLinear()
            (k_proj): TorchQuantLinear()
            (v_proj): TorchQuantLinear()
            (o_proj): TorchQuantLinear()
          )
          (mlp): LlamaMLP(
            (gate_proj): TorchQuantLinear()
            (up_proj): TorchQuantLinear()
            (down_proj): TorchQuantLinear()
            (act_fn): SiLUActivation()
          )
          (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        )
      )
      (norm): LlamaRMSNorm((2048,), eps=1e-05)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (lm_head): Linear(in_features=2048, out_features=128256, bias=False)
  )
)

In [6]:
result = model.generate("Explain the theory of relativity in simple terms.")[0]

In [7]:
print(model.tokenizer.decode(result))

<|begin_of_text|>Explain the theory of relativity in simple terms. Albert Einstein's theory of relativity revolutionized our understanding of space and time.

## Step 1


In [8]:
from datasets import load_dataset

In [9]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [10]:
quant_path = "Llama-3.2-1B-Instruct-gptqmodel-4bit"

In [11]:
calibration_dataset = load_dataset(
    "allenai/c4",
    data_files = "en/c4-train.00001-of-01024.json.gz",
    split="train",
).select(range(1024))["text"]

In [12]:
from gptqmodel import GPTQModel, QuantizeConfig

In [13]:
quant_config = QuantizeConfig(
    bits=4,
    group_size=128
)

In [14]:
import os

In [15]:
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

In [16]:
from huggingface_hub import login
login(token=HUGGINGFACE_API_KEY)

In [18]:
import os
os.environ["HF_HUB_DISABLE_XET"]="1"

In [19]:
model = GPTQModel.load(model_id,quant_config)

INFO  Estimated Quantization BPW (bits per weight): 4.2875 bpw, based on [bits: 4, group_size: 128]


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

  2025-10-24T01:16:12.446414Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://transfer.xethub.hf.co/xorbs/default/fc70bc7aaaca80289f25cc565996093c1b91929b74b45b8be4b98666b60b5a82?X-Xet-Signed-Range=bytes%3D56999338-57984345&X-Xet-Session-Id=01K89WJ7JWAQQ177G2C9RS7YYY&Expires=1761272141&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC9mYzcwYmM3YWFhY2E4MDI4OWYyNWNjNTY1OTk2MDkzYzFiOTE5MjliNzRiNDViOGJlNGI5ODY2NmI2MGI1YTgyP1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDU2OTk5MzM4LTU3OTg0MzQ1JlgtWGV0LVNlc3Npb24tSWQ9MDFLODlXSjdKV0FRUTE3N0cyQzlSUzdZWVkiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjEyNzIxNDF9fX1dfQ__&Signature=tUdAqAvNbPAfhEemMga4L~de-kzMofiqGTedyLVJ8jEh5wuZZLQn3EiGXZwBYDcqcixFVTqtAHkYxbTe5Rko4qgby7PGkIor9nA6FIZxwX83QlLLPJBrv4s7C51eH-GXiLp1PSaOhnqFjpG3nw8dApDxtB2YMEK72Ap8fXEtmjgdkwZIqbiTMeInHsyVffi3ty1ZJGHHF-RoINa0bqyFsaQ06pnUuGSg9oqdsjLEYpFNrWHUSezM6aY1mXmkRB39SRYentGSPmAv4Ibav2K2yUwyawrbYYAhTYlk85Kttn

Fetching 13 files: 100%|██████████| 13/13 [01:39<00:00,  7.62s/it]

INFO  Loader: Auto dtype (MPS or XPU): `torch.float16`                         



INFO:tokenicer.tokenicer:Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').


INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

INFO  Kernel: loaded -> `[]`                                                   


In [21]:
import os, torch
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

# keep model on CPU for quantize()
model.to("cpu")

LlamaGPTQ(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 2048)
      (layers): ModuleList(
        (0-15): 16 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): HookedLinear(in_features=2048, out_features=2048, bias=False)
            (k_proj): HookedLinear(in_features=2048, out_features=512, bias=False)
            (v_proj): HookedLinear(in_features=2048, out_features=512, bias=False)
            (o_proj): HookedLinear(in_features=2048, out_features=2048, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): HookedLinear(in_features=2048, out_features=8192, bias=False)
            (up_proj): HookedLinear(in_features=2048, out_features=8192, bias=False)
            (down_proj): HookedLinear(in_features=8192, out_features=2048, bias=False)
            (act_fn): SiLUActivation()
          )
          (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
          (post_attention_layernorm):

In [22]:
model.quantize(calibration_dataset, batch_size=1)

INFO  Packing Kernel: Auto-selection: adding candidate `TorchQuantLinear`      
INFO  Hooked Modules: Using tree based config for accurate targeting of modules
Quantizing self_attn.k_proj in layer  [0 of 15] | 0:00:52 / 0:13:52 [1/16] 6.2%

NotImplementedError: The operator 'aten::cholesky_inverse' is not currently implemented for the MPS device. If you want this op to be considered for addition please comment on https://github.com/pytorch/pytorch/issues/141287 and mention use-case, that resulted in missing op as well as commit hash 0fabc3ba44823f257e70ce397d989c8de5e362c1. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
model.save(quant_path)